<a href="https://colab.research.google.com/github/ocissor/Adversarial_Defense_Mechanism/blob/master/Cifar10Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torchvision
from torchvision import transforms,datasets
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import PIL
from PIL import Image
import shutil
from torch.utils.data.sampler import SubsetRandomSampler
import torch.backends.cudnn as cudnn
import torchvision.models as models
from tqdm import tqdm
from collections import OrderedDict

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
torch.backends.cudnn.deterministic = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = torchvision.models.resnet50(pretrained=True)

In [ ]:
for param in model.parameters():
    param.requires_grad = False 
#Classifier architecture to put on top of resnet50
fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(2048,100)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(100,10)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.fc = fc

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
epochs = 15
accuracy  = []
for epoch in tqdm(range(epochs)):
    model.train()
    print("\n Training Started!")
    for data in tqdm(train_loader):
        inputs,labels=data
        inputs = inputs.to(device)
        inputs=inputs.double()
        inputs=Variable(inputs)
        labels = labels.to(device)
        labels=Variable(labels)
        optimizer.zero_grad()
        outputs=model.forward(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
#==============================================================================================================================#
    acc = 0
    Total_images = 0
    model.eval()
    print("Starting validation after epoch number {}".format(epoch))
    for data in validation_loader:
        input,label=data
        Total_images += len(input)
        input = input.to(device)
        input=input.double()
        input=Variable(input)
        label= label.to(device)
        label=Variable(label)
        output=model.forward(input)
        print("prediction:{} || label:{}".format(output.argmax().item(),label.item()))
        if output.argmax().item() == label.item():
          acc+=1
    Accuracy = acc/Total_images
    accuracy.append(Accuracy)
    print("Accuracy on validation set after {} epoch is {}".format(epoch,Accuracy))
    l = len(accuracy)
    if len(accuracy) == 1:
        pass
    if accuracy[l-1] > accuracy[l-2]:
      model_save_name = 'Cifar10classifier.pt'
      path = F"/content/gdrive/My Drive/{model_save_name}" 
      torch.save(model.state_dict(), path)  
      print("model parameters saved!!")